In [13]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import dagshub
import mlflow
import logging
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\deban\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
26,"The script for ""Scary Movie 2"" just wasn't rea...",negative
52,This is an EXCELLENT example of early Bette Da...,positive
393,I honestly fail to understand why people love ...,negative
660,Where was this film when I was a kid? After hi...,positive
575,This third Darkman was definitely better than ...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
26,script scary movie ready go problem film blata...,negative
52,excellent example early bette davis talent pro...,positive
393,honestly fail understand people love show much...,negative
660,film kid parent split tadashi move mom live gr...,positive
575,third darkman definitely better second one sti...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    256
positive    244
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive', 'negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1,
                                       'negative': 0})
df.head()

,review,sentiment
26,script scary movie ready go problem film blata...,0
52,excellent example early bette davis talent pro...,1
393,honestly fail understand people love show much...,0
660,film kid parent split tadashi move mom live gr...,1
575,third darkman definitely better second one sti...,0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)
x = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=7)

In [ ]:
mlflow_tracking_uri = 'Provide your tracking URI here'


mlflow.set_tracking_uri(mlflow_tracking_uri)
dagshub.init(repo_owner='Provide Repo Owner Name here', repo_name='Provide Repo Name Here', mlflow=True)

mlflow.set_experiment('Logistic Regression Baseline')

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\deban\anaconda3\envs\atlas\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=5bd0cb2e-9b48-43c8-9334-f29c1b23b318&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=d7dcc0161335c7ebc6282c888fd6e2ad08ad158ef8d94ce984e30c72d05d027f




Accessing as chowdhury-ai

Initialized MLflow to track repo "chowdhury-ai/movie_sentiment_classification"

Repository chowdhury-ai/movie_sentiment_classification initialized!

2025/04/13 22:15:38 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2ca8324283724c949b4dc91001ae4bd7', creation_time=1744562738405, experiment_id='0', last_update_time=1744562738405, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [16]:
# Configuration logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()

    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.3)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000) 

        logging.info("Fitting the model...")
        model.fit(x_train, y_train)
        logging.info("Model training complete")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(x_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time-start_time:.2f} seconds.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")




    except Exception as e:
        logging.error(f"An error occured: {e}", exc_info=True)

2025-04-13 23:58:06,522 - INFO - Starting MLflow run...
2025-04-13 23:58:07,076 - INFO - Logging preprocessing parameters...
2025-04-13 23:58:08,541 - INFO - Initializing Logistic Regression model...
2025-04-13 23:58:08,541 - INFO - Fitting the model...
2025-04-13 23:58:08,561 - INFO - Model training complete
2025-04-13 23:58:08,561 - INFO - Logging model parameters...
2025-04-13 23:58:08,979 - INFO - Making predictions...
2025-04-13 23:58:08,979 - INFO - Calculating evaluation metrics...
2025-04-13 23:58:08,979 - INFO - Logging evaluation metrics...
2025-04-13 23:58:11,064 - INFO - Saving and logging the model...
2025/04/13 23:58:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-04-13 23:58:18,642 - INFO - Model training and logging completed in 11.57 seconds.
2025-04-13 23:58:18,642 - INFO - Accuracy: 0.6266666666666667
2025-04-13 23:58:18,642 - INFO -

🏃 View run respected-snail-689 at: https://dagshub.com/chowdhury-ai/movie_sentiment_classification.mlflow/#/experiments/0/runs/6c2ff954523e47a888f6ccd2a8dfeb88
🧪 View experiment at: https://dagshub.com/chowdhury-ai/movie_sentiment_classification.mlflow/#/experiments/0
